In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly
import plotly_express as px
from plotly import tools
from plotly.offline import plot
import plotly.graph_objs as go
import plotly.io as pio
from os import listdir
from os.path import isfile, join
from pathlib import Path
import country_converter as coco
from plotly.subplots import make_subplots
import ipywidgets as widgets
import os
import glob
# from floweaver import *
# %matplotlib inline

# from IPython.display import display
# from ipysankeywidget import SankeyWidget


In [3]:
# Setting working directory
os.chdir ('C:\\Users\\KarlaC\\MAT-DP\\')

# Make folders for figures
if not os.path.exists('figures'):
    os.makedirs('figures')
    
if not os.path.exists('figures\countries'):
    os.makedirs('figures\countries')
if not os.path.exists('figures\TEMBA'):
    os.makedirs('figures\TEMBA')
if not os.path.exists('outputs'):
    os.makedirs('outputs')

In [4]:
dfEc_tech_sep = pd.read_csv(r'outputs/E_matbytech_bycountry.csv')
dfEc_tech_sep_m = pd.melt(dfEc_tech_sep.drop(columns = 'Unnamed: 0'), 
                     id_vars = ['tech', 'Scenario','Country','Year'], 
                     var_name = 'Material', value_name = 'Value')
dfEc_tech_sep_m['Value'] = [x/1e12 for x in dfEc_tech_sep_m['Value']]
dfEc_tech_sep_m['Unit'] = 'Mt CO2'
dfEc_tech_sep_m.loc[dfEc_tech_sep_m['Country']=='NM','Country']= 'NA'


mat_country = pd.read_csv(r'outputs/massmat_bytech_bycountry.csv')

mat_country_m = pd.melt(mat_country, 
                     id_vars = ['tech', 'Scenario','Country','Year'], 
                     var_name = 'Material', value_name = 'Value')
mat_country_m['Value'] = [x/1000000 for x in mat_country_m['Value']]
mat_country_m['Unit'] = 'tonne' 
mat_country_m.loc[mat_country_m['Country']=='NM','Country']= 'NA'


In [5]:
def choose_scen(df, lscen, lreg):
    # Choose scenarios based on a list of the ones needed, group and make figs
    df2 = df[(df['Scenario'].isin(lscen))&(~df['Country'].isin(['UK','United Kingdom']))]
    
    # Get ISO3 codes and short names
    temba_codes = list(df2['Country'].unique())
    shorts = coco.convert(names=temba_codes, to='name_short', not_found=None)
    iso2_codes = coco.convert(names=temba_codes, to='ISO2', not_found=None)
    
    tis_d = pd.DataFrame(list(zip(temba_codes,shorts)), columns = ['Country','aCountry'])
    tis_d = tis_d.set_index('Country')['aCountry'].to_dict()
    
    ti2_d = pd.DataFrame(list(zip(shorts,iso2_codes)), columns = ['Country','ISO2'])
    ti2_d = ti2_d.set_index('Country')['ISO2'].to_dict()
    
    df2['aCountry'] = df2['Country'].map(tis_d)
    df2['ISO2'] = df2['aCountry'].map(ti2_d)
    
    if 'Material' in df2.columns:
    # Group by region and scenarios
        df2['Region'] = df2['Country'].map(lreg)
        df3 = df2.groupby(['Scenario','Region','Year','tech','Material','Unit']).sum().reset_index()

    if 'parameter' in df2.columns:
        # Group by region and scenarios
        df2['Region'] = df2['ISO2'].map(lreg)
        df3 = df2.groupby(['Scenario','Region','Year','tech','parameter','Indicator']).sum().reset_index()
    return df3

def choose_countries(df, lscen, lreg):
    # Choose countries based on a list of the ones needed
    df2 =  df[(df['Scenario'].isin(lscen))&(~df['Country'].isin(['UK','United Kingdom']))]
    
    # Get ISO3 codes and short names
    temba_codes = list(df2['Country'].unique())
    shorts = coco.convert(names=temba_codes, to='name_short', not_found=None)
    iso2_codes = coco.convert(names=temba_codes, to='ISO2', not_found=None)
    
    tis_d = pd.DataFrame(list(zip(temba_codes,shorts)), columns = ['Country','aCountry'])
    tis_d = tis_d.set_index('Country')['aCountry'].to_dict()
    
    ti2_d = pd.DataFrame(list(zip(shorts,iso2_codes)), columns = ['Country','ISO2'])
    ti2_d = ti2_d.set_index('Country')['ISO2'].to_dict()
    
    df2['aCountry'] = df2['Country'].map(tis_d)
    df2['ISO2'] = df2['aCountry'].map(ti2_d)
    
    # Group by region and scenarios
    df2['Region'] = df2['Country'].map(lreg)
    df3 = df2.groupby(['Country','Scenario','Region','Year','tech','Material','Unit']).sum().reset_index()
    return df3

# def tot_figs(df):
    
#     return fig

In [6]:
afr_regions = { 'CM':'Central Africa', 'CF':'Central Africa',
 'TD':'Central Africa' , 'CG':'Central Africa' , 
 'CD':'Central Africa', 'GQ':'Central Africa', 'GA':'Central Africa',
 'BI':'Eastern Africa','DJ':'Eastern Africa', 'ER':'Eastern Africa', 'ET':'Eastern Africa', 'KE':'Eastern Africa',
    'RW':'Eastern Africa', 'SO':'Eastern Africa', 'SD':'Eastern Africa', 'TZ':'Eastern Africa', 'UG':'Eastern Africa',
    'DZ':'Northern Africa', 'EG':'Northern Africa', 'LY':'Northern Africa', 'MR':'Northern Africa', 
    'MA':'Northern Africa', 'TN':'Northern Africa',
 'AO':'Southern Africa', 'BW':'Southern Africa', 'LS':'Southern Africa', 'MW':'Southern Africa',
    'MZ':'Southern Africa', 'NA':'Southern Africa', 'ZA':'Southern Africa', 
    'SZ':'Southern Africa', 'ZM':'Southern Africa', 'ZW':'Southern Africa',
  'BJ':'Western Africa', 'BF':'Western Africa', 'CI':'Western Africa', 'GM':'Western Africa',
    'GH':'Western Africa', 'GN':'Western Africa', 'GW':'Western Africa', 'LR':'Western Africa',
    'ML':'Western Africa', 'NE':'Western Africa', 'NG':'Western Africa', 'SN':'Western Africa',
    'SL':'Western Africa', 'TG':'Western Africa'}

## Materials and emissions by region

In [7]:
TEMBAsc = ['Reference','1.5deg', '2.0deg']
mat_TEMBA = choose_scen(mat_country_m, TEMBAsc, afr_regions)
E_TEMBA = choose_scen(dfEc_tech_sep_m, TEMBAsc, afr_regions)

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [111]:
# Making plots per scenario for materials and emissions
colors = px.colors.qualitative.Alphabet

dfs = ['mat_TEMBA', 'E_TEMBA' ]
labels = ['Material Mass [tonne]','Embodied Emissions [MtCO<sub>2</sub>]']

for d, l in zip(dfs, labels):
    data = globals()[d]
    for sc in data.Scenario.unique():
        fig = go.Figure()
        data2 = data[data['Scenario']==sc]
        
        # mat needs to divide colors by material, while emissions can use region
        if d == 'mat_TEMBA':
            data3 = data2.drop(columns = ['tech','Unit',
                                                'Region']).groupby(['Year',
                                                                    'Material']).sum().reset_index()
            col = 'Material'
        if d == 'E_TEMBA':
            data3 = data2.drop(columns = ['tech','Unit','Material']).groupby(['Year',
                                                                     'Region']).sum().reset_index()
            col = 'Region'

        for i, c in zip(data3[col].unique(), colors):
            fig.add_trace(go.Bar(x = data3[data3[col]==i].Year, 
                                 y = data3[data3[col]==i]['Value'],
                                 name = i, 
                                 marker_color = c, orientation = 'v'
                                ))

        fig.update_layout( yaxis_title =  l, 
                          template = 'simple_white+presentation', barmode='stack')
        fig.update_yaxes(showgrid=True)

        pio.write_image(fig, r'figures/{0}_{1}.pdf'.format(d,sc), width = 800, height = 800)
        pio.write_image(fig, r'figures/{0}_{1}.eps'.format(d,sc), width = 800, height = 800)
    #     plotly.offline.plot(fig, filename = r'figures/{0}_{1}.html'.format(d,sc), auto_open=False)
    #     fig.show()

In [51]:
# As subplots - cumulative values

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['mat_TEMBA', 'E_TEMBA','E_TEMBA' ]
labels = ['Material Mass [Mtonne]','Embodied Emissions [MtCO<sub>2</sub>]','Embodied Emissions [MtCO<sub>2</sub>]']
types = ['Material','Material','Region']
tf_df = [True,False,False]
            
for d, l, ty in zip(dfs, labels,types):
    data = globals()[d]
#     data['Order'] = 0
#     data.loc[data['Scenario'] == '1.5deg','Order'] = 1
#     data.loc[data['Scenario'] == '2.0deg','Order'] = 2
#     data = data.sort_values(by = 'Order', ascending = True)
#     data.drop(columns = 'Order', inplace = True)
    fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                           subplot_titles=['Reference','1.5 °C', '2.0 °C '], horizontal_spacing=0.02)
#     for sc, tf, it in zip(data.Scenario.unique(), tf_df, range(4)):
    for sc, tf, it in zip(['Reference','1.5deg', '2.0deg'], tf_df, range(4)):
        data2 = data[data['Scenario']==sc]
        
        if d == 'E_TEMBA' and ty == 'Region':
            data3 = data2.drop(columns = ['tech','Unit','Material']).groupby(['Year',
                                                                             'Region']).sum().reset_index()
            col = 'Region'
        if d == 'E_TEMBA' and ty == 'Material':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                     'Material']).sum().reset_index()
            col = 'Material'
        if d == 'mat_TEMBA':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                    'Material']).sum().reset_index()
            data3['Value'] = [x/1000000 for x in data3['Value']]
            col = 'Material'
        print(d, col, ty)    
        for i, c in zip(data3[col].unique(), colors):
            fig.add_trace(go.Bar(x = data3[data3[col]==i].Year, 
                                 y = data3[data3[col]==i]['Value'],
                                 name = i, 
                                 showlegend = tf, 
                                 marker_color = c, orientation = 'v'
                                ),
                         row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

    fig.update_layout( yaxis_title =  l, 
                      template = 'simple_white+presentation', barmode='stack',
                     legend = dict(font=dict(size = 26)))
    fig.update_yaxes(showgrid=True)
    for i in fig['layout']['annotations']:
            i['font'] = dict(size=25)

    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_subp.pdf'.format(d,ty[:3]), width = 1800, height = 1100)
    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_subp.eps'.format(d,ty[:3]), width = 1800, height = 1100)
    #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
    #     fig.show()

mat_TEMBA Material Material
mat_TEMBA Material Material
mat_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Material Material
['Reference']
E_TEMBA Region Region
['1.5deg']
E_TEMBA Region Region
['2.0deg']
E_TEMBA Region Region


In [35]:
# mat_TEMBAy = pd.pivot_table(mat_TEMBA, values = 'Value', 
#               index = ['Scenario', 'Region','tech','Material','Unit'], 
#               columns = 'Year', aggfunc=np.sum).reset_index()
# mat_TEMBAc = mat_TEMBAy
# for y in mat_TEMBA['Year'].unique():
#     if y != 2065:
#         mat_TEMBAy[y+1] = mat_TEMBAc[y+1] - mat_TEMBAc[y]
# Ymat_TEMBA = pd.melt(mat_TEMBAy, 
#                      id_vars = ['Scenario', 'Region','tech','Material','Unit'], 
#                      var_name = 'Year', value_name = 'Value')
        

In [46]:
# cap = pd.read_csv(r'outputs/TEMBA_capacities.csv')
# cap = cap[cap['parameter']=='Power Generation (Aggregate)']


In [17]:
# Calculating the non-aggregated values 
# after checking the resulting values, there is no way to get the non-aggregate ones because
# the power generated shifts in some years, e.g. gas has a lower value in 2016 than 2015
def calc_noncum(df):
    # Making pivot table
    df2 = pd.pivot_table(mat_TEMBA, values = 'Value', 
              index = ['Scenario', 'Region','tech','Material','Unit'], 
              columns = 'Year', aggfunc=np.sum).reset_index()
    dfc = df2
    # Year cum - year cum-1
    for y in df['Year'].unique():
        if y != 2065:
            df2[y+1] = dfc[y+1] - dfc[y]
    df3 = pd.melt(df2, 
                     id_vars = ['Scenario', 'Region','tech','Material','Unit'], 
                     var_name = 'Year', value_name = 'Value')
    return df3

In [18]:
mat_TEMBAy = calc_noncum(mat_TEMBA)
E_TEMBAy = calc_noncum(E_TEMBA)

In [21]:
# As subplots - non-cumulative values - NOT to be used, since the calculation is not possible by the means used

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['mat_TEMBAy', 'E_TEMBAy','E_TEMBAy' ]
labels = ['Material Mass [Mtonne]','Embodied Emissions [MtCO<sub>2</sub>]','Embodied Emissions [MtCO<sub>2</sub>]']
types = ['Material','Material','Region']
tf_df = [True,False,False]
            
for d, l, ty in zip(dfs, labels,types):
    data = globals()[d]
    fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                           subplot_titles=['Reference','1.5 °C', '2.0 °C '], horizontal_spacing=0.02)
#     for sc, tf, it in zip(data.Scenario.unique(), tf_df, range(4)):
    for sc, tf, it in zip(['Reference','1.5deg', '2.0deg'], tf_df, range(4)):
        data2 = data[data['Scenario']==sc]
        
        if d == 'E_TEMBAy' and ty == 'Region':
            data3 = data2.drop(columns = ['tech','Unit','Material']).groupby(['Year',
                                                                             'Region']).sum().reset_index()
            col = 'Region'
        if d == 'E_TEMBAy' and ty == 'Material':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                     'Material']).sum().reset_index()
            col = 'Material'
        if d == 'mat_TEMBAy':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                    'Material']).sum().reset_index()
            data3['Value'] = [x/1000000 for x in data3['Value']]
            col = 'Material'
        print(d, col, ty)    
        for i, c in zip(data3[col].unique(), colors):
            fig.add_trace(go.Bar(x = data3[data3[col]==i].Year, 
                                 y = data3[data3[col]==i]['Value'],
                                 name = i, 
                                 showlegend = tf, 
                                 marker_color = c, orientation = 'v'
                                ),
                         row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

    fig.update_layout( yaxis_title =  l, 
                      template = 'simple_white+presentation', barmode='stack',
                     legend = dict(font=dict(size = 26)))
    fig.update_yaxes(showgrid=True)
    for i in fig['layout']['annotations']:
            i['font'] = dict(size=25)

    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_ysubp.pdf'.format(d,ty[:3]), width = 1800, height = 1100)
    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_ysubp.eps'.format(d,ty[:3]), width = 1800, height = 1100)
    #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
    #     fig.show()

mat_TEMBAy Material Material
mat_TEMBAy Material Material
mat_TEMBAy Material Material
E_TEMBAy Material Material
E_TEMBAy Material Material
E_TEMBAy Material Material
E_TEMBAy Region Region
E_TEMBAy Region Region
E_TEMBAy Region Region


In [53]:
# As subplots - grouping materials w small emissions

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['mat_TEMBA', 'E_TEMBA','E_TEMBA' ]
labels = ['Material Mass [Mtonne]','Embodied Emissions [MtCO<sub>2</sub>]','Embodied Emissions [MtCO<sub>2</sub>]']
types = ['Material','Material','Region']
tf_df = [True,False,False]

colors_df = pd.DataFrame(zip(np.append(E_TEMBA['Material'].unique(),'Other'), colors), columns = ['Material','Colm'])
colors_df = colors_df.set_index('Material')['Colm'].to_dict() 
            
for d, l, ty in zip(dfs, labels,types):
    data = globals()[d]
    fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                           subplot_titles=['Reference','1.5 °C', '2.0 °C '], horizontal_spacing=0.02)
    for sc, tf, it in zip(['Reference','1.5deg', '2.0deg'], tf_df, range(4)):
        data2 = data[data['Scenario']==sc]
        
        if d == 'E_TEMBA' and ty == 'Region':
            data3 = data2.drop(columns = ['tech','Unit','Material']).groupby(['Year',
                                                                             'Region']).sum().reset_index()
            col = 'Region'
        if d == 'E_TEMBA' and ty == 'Material':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                     'Material']).sum().reset_index()
            col = 'Material'
            # Calculating the percentage per material
            tot = data3.groupby('Year').sum().reset_index().rename(columns = {'Value':'Tot'})
            data3 = data3.merge(tot, on = 'Year')
            data3['Pct'] = [x/y for x,y in zip(data3['Value'], data3['Tot'])]
            
            # Choose only those above 20% and sum the rest as name 'Other'
            data3['Material2'] = data3['Material']
            data3['Material2'] = [x if (x == 'Cast Iron'or x== 'PVC' or x=='Steel' or x== 'Aluminium') else 'Other' for x in data3['Material']]
            
            pcdata = data3[['Year','Material2','Value']].groupby(['Year',
                                                                     'Material2']).sum().reset_index()
            pcdata = pcdata.rename(columns = {'Material2':'Material'})
            data3 = pcdata
            data3 = data3.sort_values(by = ['Value'], ascending = False)
#             pcdata = data3
            
        if d == 'mat_TEMBA':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                    'Material']).sum().reset_index()
            data3['Value'] = [x/1000000 for x in data3['Value']]
            col = 'Material'
            
            
        print(d, col, ty)    
        for i, c in zip(data3[col].unique(), colors):
            if col == 'Material':
                cls = colors_df[i]
            else:
                cls = c 
            fig.add_trace(go.Bar(x = data3[data3[col]==i].Year, 
                                 y = data3[data3[col]==i]['Value'],
                                 name = i, 
                                 showlegend = tf, 
                                 marker_color = cls, orientation = 'v'
                                ),
                         row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

    fig.update_layout( yaxis_title =  l, 
                      template = 'simple_white+presentation', barmode='stack',
                     legend = dict(font=dict(size = 26)))
    fig.update_yaxes(showgrid=True)
    for i in fig['layout']['annotations']:
            i['font'] = dict(size=25)

    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_gsubp.pdf'.format(d,ty[:3]), width = 1800, height = 1100)
    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_gsubp.eps'.format(d,ty[:3]), width = 1800, height = 1100)
    #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
    #     fig.show()

mat_TEMBA Material Material
mat_TEMBA Material Material
mat_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Region Region
E_TEMBA Region Region
E_TEMBA Region Region


In [62]:
pcdata.set_index('Year')#.sort_values(by = ['Value'], ascending = False)

,Material,Value
Year,,
2015,Aluminium,0.077650
2015,Cast Iron,1.389950
2015,Other,0.330092
2015,PVC,0.626623
2015,Steel,2.416772
...,...,...
2065,Aluminium,24.462960
2065,Cast Iron,3.318527
2065,Other,7.424446


### Materials and emissions by region (with errors)

In [35]:
errorsE_bymat_bycountry = pd.read_csv(r'outputs/errorsE_bymat_bycountry.csv')

# dictionary to map tech names, since the script that generated the error data has a bug and does not show tech
und = dfEc_tech_sep[['Unnamed: 0','tech']][:15].set_index('Unnamed: 0')['tech'].to_dict()
errorsE_bymat_bycountry['tech'] = errorsE_bymat_bycountry['Unnamed: 0'].map(und)

errorsE_m = pd.melt(errorsE_bymat_bycountry.drop(columns = 'Unnamed: 0'), 
                     id_vars = ['tech', 'Scenario','Country','Year'], 
                     var_name = 'Material', value_name = 'Value')
errorsE_m['Value'] = [x/1e15 for x in errorsE_m['Value']] # should be 1e12
errorsE_m['Unit'] = 'Mt CO2'
errorsE_m.loc[errorsE_m['Country']=='NM','Country']= 'NA'

TEMBAsc = ['Reference','1.5deg', '2.0deg']
eE_TEMBA = choose_scen(errorsE_m, TEMBAsc, afr_regions)

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [43]:
# As subplots with errors

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['E_TEMBA','E_TEMBA' ]
labels = ['Embodied Emissions [MtCO<sub>2</sub>]','Embodied Emissions [MtCO<sub>2</sub>]']
types = ['Material','Region']
tf_df = [True,False,False]
            
for d, l, ty in zip(dfs, labels,types):
    data = globals()[d]
    fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                           subplot_titles=['1.5 °C', '2.0 °C ', 'Reference'], horizontal_spacing=0.02)
    for sc, tf, it in zip(data.Scenario.unique(), tf_df, range(4)):
        data2 = data[data['Scenario']==sc]
        # Error
        data2e = eE_TEMBA[eE_TEMBA['Scenario']==sc]
        
        if d == 'E_TEMBA' and ty == 'Region':
            data3 = data2.drop(columns = ['tech','Unit','Material']).groupby(['Year',
                                                                             'Region']).sum().reset_index()
            data3e = data2e.drop(columns = ['tech','Unit','Material']).groupby(['Year',
                                                                             'Region']).sum().reset_index()
            
            col = 'Region'
        if d == 'E_TEMBA' and ty == 'Material':
            data3 = data2.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                     'Material']).sum().reset_index()
            data3e = data2e.drop(columns = ['tech','Unit','Region']).groupby(['Year',
                                                                     'Material']).sum().reset_index()
            col = 'Material'
        print(d, col, ty)    
        for i, c in zip(data3[col].unique(), colors):
            fig.add_trace(go.Bar(x = data3[data3[col]==i].Year, 
                                 y = data3[data3[col]==i]['Value'],
                                 name = i, 
                                 showlegend = tf, 
                                 marker_color = c, orientation = 'v',
                                 error_y=dict(type='data', 
                                      array=data3e[data3e[col]==i]['Value'])
                                ),
                         row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

    fig.update_layout( yaxis_title =  l, 
                      template = 'simple_white+presentation', barmode='stack',
                     legend = dict(font=dict(size = 26)))
    fig.update_yaxes(showgrid=True)
    for i in fig['layout']['annotations']:
            i['font'] = dict(size=25)

    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_esubp.pdf'.format(d,ty[:3]), width = 2000, height = 1000)
    pio.write_image(fig, r'figures/TEMBA/{0}_{1}_esubp.eps'.format(d,ty[:3]), width = 2000, height = 1000)
    #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
#     fig.show()

E_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Material Material
E_TEMBA Region Region
E_TEMBA Region Region
E_TEMBA Region Region


In [ ]:
# Making df fit for Sankey
df_E = E_TEMBA.rename(columns = {'tech':'Source'})
df_E['Target'] = 'Embodied emissions'
df_M = mat_TEMBA.rename(columns = {'Material':'Target','tech':'Source'})
df_M['Material'] = df_M['Target']
df = pd.DataFrame(columns = ['Material','Source','Target','Value','Unit','Year','Scenario','Region'])

df = df.append(df_E)
df = df.append(df_M)
df.to_csv(r'outputs/TEMBA_SANKEY_data.csv', index = False)

## Materials and emissions by country

In [54]:
TEMBAsc = ['Reference','1.5deg', '2.0deg']
mat_TEMBA_byc = choose_countries(mat_country_m, TEMBAsc, afr_regions)
E_TEMBA_byc = choose_countries(dfEc_tech_sep_m, TEMBAsc, afr_regions)

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [56]:
# As subplots

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['mat_TEMBA_byc', 'E_TEMBA_byc' ]
labels = ['Material Mass [tonne]','Embodied Emissions [MtCO<sub>2</sub>]']
tf_df = [True,False,False]

countries = list(mat_TEMBA_byc['Country'].unique())
cshort = coco.convert(names=countries, to='name_short', not_found=None)
        
for d, l in zip(dfs, labels):
    for cy, i2 in zip(cshort, countries):
        print(cy)
        if not os.path.exists('figures/TEMBA/{}'.format(cy)):
            os.makedirs('figures/TEMBA/{}'.format(cy))

        data = globals()[d][globals()[d]['Country']==i2]
        fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                                   subplot_titles=['Reference','1.5 °C', '2.0 °C '], horizontal_spacing=0.02)

        for sc, tf, it in zip(['Reference','1.5deg', '2.0deg'], tf_df, range(4)):
            data2 = data[data['Scenario']==sc]

            # mat needs to divide colors by material, while emissions can use region
            if d == 'mat_TEMBA_byc':
                data3 = data2.drop(columns = ['tech','Unit', 'Scenario',
                                              'Country','Region']).groupby(['Year',
                                                                        'Material']).sum().reset_index()
                col = 'Material'
                for i, c in zip(data3[col].unique(), colors):
                    fig.add_trace(go.Bar(x = data3[data3[col]==i].Year, 
                                         y = data3[data3[col]==i]['Value'],
                                         name = i, 
                                         showlegend = tf, 
                                         marker_color = c, orientation = 'v'
                                        ),
                                 row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))
                
            if d == 'E_TEMBA_byc':
                data3 = data2.drop(columns = ['tech','Unit','Scenario',
                                              'Region','Material']).groupby(['Year']).sum().reset_index()
                col = 'Country'
                fig.add_trace(go.Bar(x = data3.Year, 
                                 y = data3['Value'],
                                 name = col, 
                                 showlegend = False, 
                                 marker_color = c, orientation = 'v'
                                ),
                         row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

        fig.update_layout( yaxis_title =  l, 
                          template = 'simple_white+presentation', barmode='stack',
                         legend = dict(font=dict(size = 26)))

        for i in fig['layout']['annotations']:
                i['font'] = dict(size=25)
        fig.update_yaxes(showgrid=True)

        pio.write_image(fig, r'figures/TEMBA/{0}/{1}_{2}_subp.pdf'.format(cy,i2,d), width = 1800, height = 1100)
        pio.write_image(fig, r'figures/TEMBA/{0}/{1}_{2}_subp.eps'.format(cy,i2,d), width = 1800, height = 1100)
        #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
        #     fig.show()

Angola
Burkina Faso
Burundi
Benin
Botswana
DR Congo
Central African Republic
Congo Republic
Cote d'Ivoire
Cameroon
Djibouti
Algeria
Egypt
Eritrea
Ethiopia
Gabon
Ghana
Gambia
Guinea
Equatorial Guinea
Guinea-Bissau
Kenya
Liberia
Lesotho
Libya
Morocco
Mali
Mauritania
Malawi
Mozambique
Namibia
Niger
Nigeria
Rwanda
Sudan
Sierra Leone
Senegal
Somalia
Eswatini
Chad
Togo
Tunisia
Tanzania
Uganda
South Africa
Zambia
Zimbabwe
Angola
Burkina Faso
Burundi
Benin
Botswana
DR Congo
Central African Republic
Congo Republic
Cote d'Ivoire
Cameroon
Djibouti
Algeria
Egypt
Eritrea
Ethiopia
Gabon
Ghana
Gambia
Guinea
Equatorial Guinea
Guinea-Bissau
Kenya
Liberia
Lesotho
Libya
Morocco
Mali
Mauritania
Malawi
Mozambique
Namibia
Niger
Nigeria
Rwanda
Sudan
Sierra Leone
Senegal
Somalia
Eswatini
Chad
Togo
Tunisia
Tanzania
Uganda
South Africa
Zambia
Zimbabwe


## Emissions for all countries in one graph

In [57]:
# Same as section 2, but here in case I only rerun this section
TEMBAsc = ['Reference','1.5deg', '2.0deg']
mat_TEMBA_byc = choose_countries(mat_country_m, TEMBAsc, afr_regions)
E_TEMBA_byc = choose_countries(dfEc_tech_sep_m, TEMBAsc, afr_regions)

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [58]:
# As subplots

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Vivid

dfs = ['mat_TEMBA_byc', 'E_TEMBA_byc' ]
labels = ['Material Mass [tonne]','Embodied Emissions [MtCO<sub>2</sub>]']
tf_df = pd.DataFrame(columns = ['Country'],data = E_TEMBA_byc['Country'].unique())
tf_df['Val'] = False
tf_df.loc[tf_df['Country']=='AO', 'Val'] = True

countries = list(mat_TEMBA_byc['Country'].unique())
cshort = coco.convert(names=countries, to='name_short', not_found=None)
        
# for d, l in zip(dfs, labels):
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,  shared_yaxes=True, \
                               subplot_titles=['Reference','1.5 °C', '2.0 °C '], vertical_spacing=0.06,
                    x_title = 'Country',
                   y_title = 'Cumulative Embodied Emissions [MtCO<sub>2</sub>]' )

for sc, it in zip(['Reference','1.5deg', '2.0deg'], range(4)):
    data2 = E_TEMBA_byc[E_TEMBA_byc['Scenario']==sc]

    data3 = data2.drop(columns = ['tech','Unit','Scenario',
                                  'Region','Material']).groupby(['Country','Year']).sum().reset_index()
    data41 = data3[data3['Year']==2015]
    data42 = data3[data3['Year']==2025]
    data43 = data3[data3['Year']==2035]
    data44 = data3[data3['Year']==2045]
    data45 = data3[data3['Year']==2055]
    data5 = data3[data3['Year']==2065]
    data6 = data5
    data42['Value'] = [a-b for a,b in zip(data42['Value'], data41['Value'])]
    data43['Value'] = [a-(b+c) for a,b,c in zip(data43['Value'],data42['Value'], data41['Value'])]
    data44['Value'] = [a-(b+c+d) for a,b,c,d in zip(data44['Value'],data43['Value'],data42['Value'], data41['Value'])]
    data45['Value'] =  [a-(b+c+d+e) for a,b,c,d,e in zip(data45['Value'],data44['Value'],data43['Value'],data42['Value'], data41['Value'])] 
    data6['Value'] = [a-(b+c+d+e+f) for a,b,c,d,e,f in zip(data5['Value'], data41['Value'],data42['Value'],data43['Value'],data44['Value'],data45['Value'])]
    data7 = data41
    for iy in range(4):
        data7 = data7.append(globals()['data4'+str(iy+2)])
    data7 = data7.append(data6)
    totals = data3[data3['Year']==2065].rename(columns = {'Value':'Total'})
    col = 'Country'
    for cy, i2, tf in zip(cshort, totals.sort_values(by ='Total', ascending = False)['Country'], tf_df['Val']):
        if sc == 'Reference':
            tf = tf
        else:
            tf = False
        tx = list(totals[totals['Country']==i2].Total)
        tx = [np.round(x,1) for x in tx]
        for c, y, ry in zip(colors,[2015,2025,2035,2045,2055,2065], ['Until 2015','2016-2025','2026-2035','2036-2045','2046-2055','2056-2065']):
            fig.add_trace(go.Bar(x = data7[(data7['Country']==i2)&(data7['Year']==y)].Country, 
                             y = data7[(data7['Country']==i2)&(data7['Year']==y)]['Value'],
                             name = ry, 
                                 text = tx,
                             showlegend = tf, 
                             marker_color = c, orientation = 'v'
                            ),
                     row= int(rc.iloc[it][1]), col =int(rc.iloc[it][0]))

fig.update_layout( 
                  template = 'simple_white+presentation', barmode='stack',
                 legend = dict(font=dict(size = 26)))

for i in fig['layout']['annotations']:
        i['font'] = dict(size=25)
fig.update_yaxes(showgrid=True)

pio.write_image(fig, r'figures/TEMBA/CumCountryEm_subp.pdf', width = 1850, height = 1100)
pio.write_image(fig, r'figures/TEMBA/CumCountryEm_subp.eps', width = 1850, height = 1100)
    #     plotly.offline.plot(fig, filename = r'figures/TEMBA/CumCountryEm_subp.html', auto_open=False)
# fig.show()

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\KarlaC\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

## Total capacity per region

In [ ]:
results_df = pd.read_csv(r'C:/Users/KarlaC/MAT-DP/outputs/TEMBA_capacities.csv')
# Capacities in GW

results_df.loc[results_df['country']=='NM','country']= 'NA'
results_df = results_df.rename(columns = {'scenario':'Scenario', 'country':'Country'})

In [ ]:

# Get ISO3 codes and short names
temba_codes = list(results_df['Country'].unique())
shorts = coco.convert(names=temba_codes, to='name_short', not_found=None)
iso2_codes = coco.convert(names=temba_codes, to='ISO2', not_found=None)

tis_d = pd.DataFrame(list(zip(temba_codes,shorts)), columns = ['Country','aCountry'])
tis_d = tis_d.set_index('Country')['aCountry'].to_dict()

ti2_d = pd.DataFrame(list(zip(shorts,iso2_codes)), columns = ['Country','ISO2'])
ti2_d = ti2_d.set_index('Country')['ISO2'].to_dict()

results_df['aCountry'] = results_df['Country'].map(tis_d)
results_df['ISO2'] = results_df['aCountry'].map(ti2_d)

# Group by region and scenarios
results_df['Region'] = results_df['Country'].map(afr_regions)


In [49]:
sorted(data.Scenario.unique())

['1.5deg', '2.0deg', 'Reference']

In [ ]:
# As subplots - totals

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['New power generation capacity (Aggregate)',
       'Power Generation Capacity (Aggregate)']
labels = ['New power generation capacity (Aggregate) [GW]',
       'Power Generation Capacity (Aggregate) [GW]']
tf_df = [True,False,False]

countries = list(results_df['Country'].unique())
cshort = coco.convert(names=countries, to='name_short', not_found=None)

for d, l in zip(dfs, labels):
    data = results_df[results_df['parameter']==d]
    fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                        subplot_titles=sorted(['1.5 °C', '2.0 °C ', 'Reference']), horizontal_spacing=0.02)

    for sc, tf, it in zip(sorted(data.Scenario.unique()), tf_df, range(4)):

        data2 = data[data['Scenario']==sc]

        # mat needs to divide colors by material, while emissions can use region
        data3 = data2.drop(columns = ['Scenario','parameter',
                                      'Country','Region']).groupby(['Year', 'tech']).sum().reset_index()
        for i, c in zip(data3['tech'].unique(), colors):
            fig.add_trace(go.Bar(x = data3[data3['tech']==i].Year, 
                                 y = data3[data3['tech']==i]['Value'],
                                 name = i, 
                                 showlegend = tf, 
                                 marker_color = c, orientation = 'v'
                                ),
                         row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

        fig.update_layout( yaxis_title =  l, 
                          template = 'simple_white+presentation', barmode='stack',
                         legend = dict(font=dict(size = 26)))

        for i in fig['layout']['annotations']:
                i['font'] = dict(size=25)
        fig.update_yaxes(showgrid=True)
        pio.write_image(fig, r'figures/TEMBA/Capacity_{}_subp.pdf'.format(d[0:3]), width = 1800, height = 1000)
        pio.write_image(fig, r'figures/TEMBA/Capacity_{}_subp.eps'.format(d[0:3]), width = 1800, height = 1000)
        #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
        #     fig.show()

In [ ]:
# As subplots by country

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['New power generation capacity (Aggregate)',
       'Power Generation Capacity (Aggregate)']
labels = ['New power generation capacity (Aggregate) [GW]',
       'Power Generation Capacity (Aggregate) [GW]']
tf_df = [True,False,False]

countries = list(results_df['Country'].unique())
cshort = coco.convert(names=countries, to='name_short', not_found=None)

for d, l in zip(dfs, labels):
    for cy, i2 in zip(cshort, countries):
        print(cy)
        if not os.path.exists('figures/TEMBA/{}'.format(cy)):
            os.makedirs('figures/TEMBA/{}'.format(cy))

        data = results_df[(results_df['parameter']==d)&(results_df['Country']==i2)]
        fig = make_subplots(rows=1, cols=3, shared_xaxes=True,  shared_yaxes=True, \
                            subplot_titles=sorted(['1.5 °C', '2.0 °C ', 'Reference']), horizontal_spacing=0.02)

        for sc, tf, it in zip(sorted(data.Scenario.unique()), tf_df, range(4)):
            data2 = data[data['Scenario']==sc]

            # mat needs to divide colors by material, while emissions can use region
            data3 = data2.drop(columns = ['Scenario','parameter',
                                          'Country','Region']).groupby(['Year', 'tech']).sum().reset_index()
            for i, c in zip(data3['tech'].unique(), colors):
                fig.add_trace(go.Bar(x = data3[data3['tech']==i].Year, 
                                     y = data3[data3['tech']==i]['Value'],
                                     name = i, 
                                     showlegend = tf, 
                                     marker_color = c, orientation = 'v'
                                    ),
                             row= int(rc.iloc[it][0]), col =int(rc.iloc[it][1]))

        fig.update_layout( yaxis_title =  l, 
                          template = 'simple_white+presentation', barmode='stack',
                         legend = dict(font=dict(size = 26)))

        for i in fig['layout']['annotations']:
                i['font'] = dict(size=25)
        fig.update_yaxes(showgrid=True)
        pio.write_image(fig, r'figures/TEMBA/{0}/Capacity_{1}_{2}_subp.pdf'.format(cy,i2,d[0:3]), width = 1800, height = 1000)
        pio.write_image(fig, r'figures/TEMBA/{0}/Capacity_{1}_{2}_subp.eps'.format(cy,i2,d[0:3]), width = 1800, height = 1000)
        #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
        #     fig.show()

## Total employment by region

In [151]:
jobs_forplot = pd.read_csv(r'outputs/jobs_forplot.csv')
jobs_forplot.rename(columns ={'scenario':'Scenario'}, inplace = True)
jobs = choose_scen(jobs_forplot.drop(columns = ['Unnamed: 0','country','ISO3']), TEMBAsc, afr_regions)

In [137]:
a = jobs[(jobs['Scenario']=='2.0deg')&(jobs['Indicator']!='Capacity')&(jobs['parameter']=='New power generation capacity (Aggregate)')]
a[a['Value']>0]['tech'].unique()

array(['Solar PV', 'Wind (Offshore)', 'Nuclear', 'Solar CSP', 'Biomass',
       'Geothermal'], dtype=object)

In [155]:
# Scenario subplots

# for choosing row and col
rc = pd.DataFrame()
for i in range(1,2):
    for j in range(1,4):
#         print(i,j)
        rc = rc.append([[i,j]])
rc.columns = ['rn','cn']
rc= rc.reset_index().drop(columns = 'index')

colors = px.colors.qualitative.Alphabet

dfs = ['New power generation capacity (Aggregate)',
       'Power Generation Capacity (Aggregate)']
labels = ['New power generation capacity (Aggregate) [GW]',
       'Power Generation Capacity (Aggregate) [GW]']
tf_df = [False,False,True]

countries = list(results_df['Country'].unique())
cshort = coco.convert(names=countries, to='name_short', not_found=None)

data = jobs[(jobs['parameter']=='New power generation capacity (Aggregate)')&(jobs['Indicator']!='Capacity')]
data = data[data['Value']>0]
for spli in ['tech','Region','Indicator']:
#     del t
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,  shared_yaxes=True, \
                        subplot_titles=sorted(['1.5 °C', '2.0 °C ', 'Reference']), vertical_spacing=0.08, 
                        x_title = 'Year', y_title = 'Number of new jobs (M)')
    for sc, tf, it in zip(sorted(data.Scenario.unique()), tf_df, range(4)):
            data2 = data[data['Scenario']==sc]
            # Average number of jobs
#             data4 = data2.drop(columns = ['Scenario','parameter',
#                               'Region','Indicator','tech']).groupby(['Year']).sum().reset_index()
#             ave = np.mean(data4['Value']/1000000)
#             print(ave)
            
            # mat needs to divide colors by material, while emissions can use region
            if spli == 'tech':
                data3 = data2.drop(columns = ['Scenario','parameter',
                              'Region','Indicator']).groupby(['Year','tech']).sum().reset_index()
            if spli == 'Region':
                data3 = data2.drop(columns = ['Scenario','parameter',
                              'tech','Indicator']).groupby(['Year', 'Region']).sum().reset_index()
            if spli == 'Indicator':
                data3 = data2.drop(columns = ['Scenario','parameter',
                              'tech','Region']).groupby(['Year', 'Indicator']).sum().reset_index()
            data3['Value'] = [x/1000000 for x in data3['Value']]
            print(data3[spli].unique())
            for t, co  in zip(data3[spli].unique(), colors):
    #                     print(t)
                fig.add_trace(go.Scatter(x = data3[data3[spli]==t].Year, 
                                         y = data3[data3[spli]==t].Value,
                                         mode = 'lines',
                                         name = t ,
                                         showlegend = tf,
                                         line_color = co, 
                                         stackgroup='one'),
                              row= int(rc.iloc[it][1]), col =int(rc.iloc[it][0])
                             )
                fig.update_layout(template = 'simple_white+presentation',
    #                               barmode='stack'
                                 )
            
    for i in fig['layout']['annotations']:
            i['font'] = dict(size=25)
    fig.update_yaxes(showgrid=True)
    pio.write_image(fig, r'figures/TEMBA/Jobs{}_subp.pdf'.format(spli), width = 1800, height = 1200)
    pio.write_image(fig, r'figures/TEMBA/Jobs{}_subp.eps'.format(spli), width = 1800, height = 1200)

            #     plotly.offline.plot(fig, filename = r'figures/TEMBA/{}_subp.html'.format(d), auto_open=False)
    #         fig.show()
    
    #             if not data3.empty:
#                 for t, co  in zip(data3[typ].unique(), colors):
#                     for i, d in zip(data3['Indicator'].unique(),['solid','dash','dot']):
#                         fig.add_trace(go.Scatter(x = data3[(data3[typ]==t)&(data3['Indicator']==i)].Year, 
#                                                  y = data3[(data3[typ]==t)&(data3['Indicator']==i)].Value,
#                                                  mode = 'lines',
#                                                  name = '{}- {}'.format(t,i) ,
#                                                  line = dict(dash = d),
#                                                  line_color = co, 
#                                                  stackgroup='one'),
#                                       row= int(rc.iloc[it][1]), col =int(rc.iloc[it][0])
#                                      )

1.5534324378652662
['Biomass' 'Coal' 'Gas' 'Geothermal' 'Hydro' 'Solar CSP' 'Solar PV'
 'Wind (Offshore)' 'Nuclear']
1.5221807479152023
['Biomass' 'Coal' 'Gas' 'Geothermal' 'Hydro' 'Solar CSP' 'Solar PV'
 'Wind (Offshore)' 'Nuclear']
1.6098246104793303
['Biomass' 'Coal' 'Gas' 'Geothermal' 'Hydro' 'Solar CSP' 'Solar PV'
 'Wind (Offshore)' 'Nuclear']
1.5534324378652662
['Central Africa' 'Eastern Africa' 'Northern Africa' 'Southern Africa'
 'Western Africa']
1.5221807479152023
['Central Africa' 'Eastern Africa' 'Northern Africa' 'Southern Africa'
 'Western Africa']
1.6098246104793303
['Central Africa' 'Eastern Africa' 'Northern Africa' 'Southern Africa'
 'Western Africa']
1.5534324378652662
['Construction and Installation (local jobs)' 'Manufacturing (local jobs)']
1.5221807479152023
['Construction and Installation (local jobs)' 'Manufacturing (local jobs)']
1.6098246104793303
['Construction and Installation (local jobs)' 'Manufacturing (local jobs)']


In [ ]:
# colors = px.colors.qualitative.Vivid

# for c in jobs_forplot['Country'].unique():
#     for sc in jobs_forplot[jobs_forplot['Country']==c]['scenario'].unique():
    
#         data = jobs_forplot[(jobs_forplot['Country']==c)&\
#                             (jobs_forplot['scenario']==sc)&\
#                             (jobs_forplot['parameter']=='New power generation capacity (Aggregate)')&\
#                             (jobs_forplot['Indicator']!='Capacity')]
#         data = data[data['Value']>0]
#         if not data.empty:
#             fig = go.Figure()
# #             print(c, sc)
#             for t, co  in zip(data['tech'].unique(), colors):
#                 for i, d in zip(data['Indicator'].unique(),['solid','dash','dot']):# ('circle','square','diamond')):
#                     fig.add_trace(go.Scatter(x = data[(data['tech']==t)&(data['Indicator']==i)].Year, 
#                                              y = data[(data['tech']==t)&(data['Indicator']==i)].Value,
#                                              mode = 'lines',
#                                              name = '{}- {}'.format(t,i) ,
# #                                              marker = dict(symbol=m),
# #                                              marker_color = co,
#                                              line = dict(dash = d),
#                                              line_color = co,
#                                              stackgroup='one')
#                                  )
#             fig.update_layout(xaxis_title = 'Year', yaxis_title = 'Number of jobs',
#                               template = 'simple_white+presentation',
# #                               barmode='stack'
#                              )
# #             fig.show()            
#             pio.write_image(fig, r'figures/countries/{0}_{1}_employment.pdf'.format(c,sc[:3]), width = 1500, height = 1000)
#             pio.write_image(fig, r'figures/countries/{0}_{1}_employment.eps'.format(c,sc[:3]), width = 1500, height = 1000)